This document goes over how to extract information for machine learning models, with the following insights:

- What features in the data did the model think are most important?
- For any single prediction from a model, how did each feature in the data affect that particular prediction?
- How does each feature affect the model's predictions in a big-picture sense (what is its typical effect when considered over a large number of possible predictions)?

# Feature Engineering

This is usually the most effective way to improve model accuracy. This is done by repeatedly creating new features using transformations of the raw data or features that were previously created.

### Permutation Importance
Finding out which features have the greatest impact on predictions is a concept called __feature importance__. While there are many ways to find this answer, one of the most common and understood methods is permutation importance. 

The process is as follows:
1. Get a trained model
2. Shuffle the values in a single column, make predictions using the resulting dataset and use these predictions with the true target values to calculate how much the loss function suffered from shuffling. That performance deterioration measures the importance of the variable you just shuffled.
3. Return the data to the original order and repeat step 2 with the next column in the data set until each column has been shuffled.

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

# Make a small change to the code below to use in this problem. 
perm = PermutationImportance(first_model, random_state=1).fit(val_X, val_y)

# uncomment the following line to visualize your results
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

#repeat for following columns

### Partial Dependence Plots

Feature importance shows *what* variables most affect predictions, where partial dependence plots show *how* a feature affects predictions. Some example use cases where this can provide answers:

- Controlling for all other house features, what impact do longitude and latitude have on home prices? To restate this, how would similarly sized houses be priced in different areas?
- Are predicted health differences between two groups due to differences in their diets, or due to some other factor?

For example, if we are searching for the PDP for who will win "man of the match" in a football game, we might plot several graphs all with y-axis being the probability of receiving the award but with each graph analyzing a different criteria. For example: goals scored, km run, passes made, etc.

In [ ]:
# pdp_dist = pdp.pdp_isolate(model=my_model, dataset=my_df, model_features=['X1', 'X2'], feature='X1')
# pdp.pdp_plot(pdp_dist, 'X1')

### SHAP

While we can use techniques to extract general data from a model, we can also break down a prediction to show the impact of each feature. Example use cases: 

- A model says a bank shouldn't loan someone money, and the bank is legally required to explain the basis for each loan rejection
- A healthcare provider wants to identify what factors are driving each patient's risk of some disease so they can directly address those risk factors with targeted health interventions

SHAP is an acronym from SHapley Additive exPlanations, and the values interpret the impact of having a certain value for a given feature in comparison to the prediction we'd make if that feature took some baseline value. The equation for calculating this is as follows:

    sum(SHAP values for all features) = pred_for_group - pred_for_baseline_values
    
SHAP values are plotted as such:

In [3]:
from IPython.display import Image
Image(url='https://i.imgur.com/JVD2U7k.png')

This graph is a continuation of the soocer example used above. An explanation is providing below:

We predicted 0.7, whereas the base_value is 0.4979. Feature values causing increased predictions are in pink, and their visual size shows the magnitude of the feature's effect. Feature values decreasing the prediction are in blue. The biggest impact comes from Goal Scored being 2. Though the ball possession value has a meaningful effect decreasing the prediction.

If you subtract the length of the blue bars from the length of the pink bars, it equals the distance from the base value to the output.

In [ ]:
import shap  # package used to calculate Shap values

# Create object that can calculate shap values
explainer = shap.TreeExplainer(my_model)

# Calculate Shap values
shap_values = explainer.shap_values(data_for_prediction)

shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], data_for_prediction)

### Future Data Collection

Deciding to embark on new data collection can be an expensive endeavour, especially for large companies. Having a good understanding of the features used in your model can direct your conclusions to which features would be worthwhile to collect.